In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("../clean_data/df_3to1.csv")
# Warning: for TAU,PTAU, ABETA -> fine, I'm not using them
df.drop(['ABETA', 'TAU', 'PTAU'], axis=1, inplace=True)

/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_7614/1384790660.py:1: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../clean_data/df_3to1.csv")


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
def d_overview(df):
    print(f"Dimension: {df.shape}")
    output = df.apply(lambda x: (x.dtype,x.isna().mean(),x.unique()), axis=0).T
    output.columns = ["dtype", "NaN_precentage","unique"]
    return output.sort_values(by="NaN_precentage", ascending=False)

### Note:

- make the data balanced by identifying MCI-> AD from timepoint that not start from bl

- use ADNI1 & GO as training, ADNI2 as testing

In [4]:
display(d_overview(df))

Dimension: (11364, 88)


,dtype,NaN_precentage,unique
PIB,float64,0.980465,"[nan, 2.3575, 2.1275, 2.0725, 2.19, 2.27, 2.35..."
FBB,float64,0.955209,"[nan, 1.0714, 1.0372, 0.9902, 0.9959, 0.9602, ..."
TAU_UPENN,float64,0.811246,"[nan, 61.4, 76.5, 41.8, 49.4, 116.0, 120.0, 12..."
PTAU_UPENN,float64,0.809222,"[nan, 20.4, 21.1, 12.9, 13.5, 67.2, 40.5, 28.5..."
ABETA_UPENN,float64,0.809134,"[nan, 131.0, 137.0, 256.0, 246.0, 116.0, 136.0..."
AV45,float64,0.731345,"[nan, 1.3496, 0.9804, 0.9851, 0.994, 0.9416, 0..."
OTHER,float64,0.69113,"[nan, 0.0, 1.0]"
DPMOOD,float64,0.69069,"[nan, 0.0, 1.0]"
INSOMN,float64,0.690602,"[nan, 1.0, 0.0]"
ABDOMN,float64,0.690602,"[nan, 0.0, 1.0]"


### Check baseline MCI progression

In [64]:
df_prog = df[["RID","DX","VISCODE","ORIGPROT"]]
print("Baseline count:")
df_prog["DX"][df_prog.VISCODE==0].value_counts().sort_index()
display(pd.crosstab(df_prog["DX"], df_prog["VISCODE"]))
start_AD = df_prog["RID"][(df_prog["VISCODE"]==0) & (df_prog["DX"]==3)]
start_MCI = df_prog["RID"][(df_prog["VISCODE"]==0) & (df_prog["DX"]==2)]
start_CN = df_prog["RID"][(df_prog["VISCODE"]==0) & (df_prog["DX"]==1)]
df_cn = df_prog.loc[(df_prog["VISCODE"].isin([0,12,24,36,48,60,72]))&         # keep time point 0,12,24
            (df_prog["RID"].isin(start_CN))             # rm start with AD
           ] 
df_mci = df_prog.loc[(df_prog["VISCODE"].isin([0,12,24,36]))&         # keep time point 0,12,24
            (df_prog["RID"].isin(start_MCI))             # rm start with AD
           ]  
display(pd.crosstab(df_mci["DX"],df_mci["VISCODE"]))
pd.crosstab(df_cn["DX"],df_cn["VISCODE"])

Baseline count:


VISCODE,0,6,12,18,24,30,36,42,48,54,60,66,72,78,84,90,96,102,108,114,120,126,132,138,144,150,156,162,168,174,180,186,192,198
DX,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,892,491,493,13,577,30,251,10,350,30,138,51,164,43,101,36,87,20,50,19,49,9,26,4,11,5,17,7,0,1,10,4,0,1
2,1096,770,801,231,549,10,421,2,283,13,161,23,117,31,104,37,72,28,54,14,46,12,20,12,13,7,5,1,5,3,3,2,2,0
3,413,355,418,97,382,9,207,2,145,1,88,13,73,11,53,10,47,6,35,4,19,6,7,6,6,5,4,2,0,0,1,0,1,0


VISCODE,0,12,24,36
DX,,,,
1,0,30,39,42
2,1096,778,512,392
3,0,109,191,188


VISCODE,0,12,24,36,48,60,72
DX,,,,,,,
1,892,463,538,209,316,117,142
2,0,20,34,29,36,21,29
3,0,0,3,6,3,2,7


### Create subset

- identify patients start with MCI -> keep at MCI or turn to AD after 12 months

- outcome variable: "progress" as 0 if DX keep at 2 or 1, outcome as 1 if DX turn to 3

- patients 230, 1098, 4365, 4225,  984, 1190 are CN -> AD. Include them in the future because they are good target for early diagnosis but didn't identified MCI



In [120]:
pd.Series([True,False,True]).idxmax()

0

In [170]:
def keep_two_rows(group):
    """
    to find the two rows in patients start with CN but turn to CN or AD
    """
    group = group.reset_index(drop=True)
    first_condition = ((group["DX"]==3) & (group["lag_mo"]==12))
    if any(first_condition):
        idx2 = (first_condition).idxmax()
    elif any(group["lag_mo"]==12):      # no DX==3
        idx2 = (group["lag_mo"]==12).idxmax()
    else:
        return None
    rows_to_keep = group.iloc[idx2-1:idx2+1] # the timepoint & the previous time before 12 mo
    return rows_to_keep

In [202]:
df_prog = df[["RID","DX","VISCODE"]]
## start with MCI:
start_MCI = df_prog["RID"][(df_prog["VISCODE"]==0) & (df_prog["DX"]==2)]
mci_bl = df.loc[(df_prog["VISCODE"]==0) & (df_prog["DX"]==2)]                             # MCI at baseline
mci_after_12 = df_prog.loc[((df_prog["RID"].isin(start_MCI)) & (df_prog["VISCODE"]==12))] # MCI at baseline after 12 mo
mci_subset = pd.merge(mci_bl, mci_after_12, on="RID", how="inner")
# mci_subset = mci_subset.groupby('RID').filter(lambda x: (len(x) > 1))  # filter out the ones with one timepoint
# mci_subset.loc[mci_subset.VISCODE==12]
mci_subset["progress"] = (mci_subset.DX_y - mci_subset.DX_x == 1).astype(int)    # outcome
mci_subset.drop(['DX_x','DX_y', 'VISCODE_x','VISCODE_y'], axis=1, inplace=True)
mci_subset
## start with CN:
cn_bl = (df_prog["VISCODE"]==0) & (df_prog["DX"]==1)
cn_subset = df.loc[(df_prog["VISCODE"]!=0) & (df_prog["RID"].isin(start_CN))  # followup of val start with CN
                        & (df_prog["DX"].isin([2,3]))]                        # the ones turn to MCI|AD
cn_subset.sort_values(["RID", "VISCODE"], inplace=True)
cn_subset["lag_mo"] = cn_subset.groupby('RID',group_keys=False)['VISCODE'].apply(lambda x: x - x.shift(1))
cn_subset = cn_subset.groupby('RID').filter(lambda x: (len(x) > 1)) # filter out the ones with one point
cn_subset = cn_subset.groupby('RID').apply(keep_two_rows).reset_index(drop=True)   # keep 2 rows for 1 RID
cn_subset_bl = cn_subset.iloc[::2]  # bl
cn_subset_12 = cn_subset.iloc[1::2]  # after 12 mo
# df_prog.loc[df_prog.RID.isin(cn_subset_bl.RID[cn_subset_bl.DX==3])]  # special cases jump from 1 to 3
# remove the special cases temporarily:
cn_subset_bl = cn_subset_bl.loc[~cn_subset_bl.RID.isin([230, 1098, 4365, 4225,  984, 1190])].reset_index(drop=True) 
cn_subset_12 = cn_subset_12.loc[~cn_subset_12.RID.isin([230, 1098, 4365, 4225,  984, 1190]),["RID", "DX"]].reset_index(drop=True)
cn_subset_bl["progress"] = (cn_subset_12.DX - cn_subset_bl.DX)
cn_subset_bl.drop(["lag_mo", "DX", "VISCODE"], axis=1, inplace=True)
cn_subset_bl
df_mci = pd.concat([mci_subset, cn_subset_bl]) 
df_mci.progress.value_counts()


/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_7614/2444618567.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_subset.sort_values(["RID", "VISCODE"], inplace=True)
/var/folders/bm/s0j8htzs7tn4d1t__749xglm0000gn/T/ipykernel_7614/2444618567.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cn_subset["lag_mo"] = cn_subset.groupby('RID',group_keys=False)['VISCODE'].apply(lambda x: x - x.shift(1))


0    857
1    137
Name: progress, dtype: int64

In [203]:
df_mci.to_csv('../clean_data/df_mci.csv', index=False)